In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();
using LinearAlgebra
using JLD2
using Test
using Statistics
const resfile = joinpath(@__DIR__, "q1.jld2")

LoadError: AssertionError: sourcepath !== nothing

In [2]:
import Pkg; Pkg.add("Plots")

# Question 1: Implicit Integrators (25 pts)
In this question we'll be taking a deeper look into implicit integration techniques, and some of the advantages they bring.

## Part (a): Implicit Midpoint (3 pts)
Implement implicit midpoint integration for the simple pendulum with $m = l = 1$ and $g = 9.81$:

Recall from lecture that implicit integration is defined by the following formula:
$$ x_{n+1} = x_n + h f\left(\frac{x_{n+1} + x_n}{2}\right) $$

In order to use an implicit integrator, we need to solve a root-finding problem using Newton's method. For convenience, we'll define a function `implicit_midpoint` that outputs 
the residual error of our integrator:

$$ x_n + h f\left(\frac{x_{n+1} + x_n}{2}\right) - x_{n+1} = 0 $$


**NOTE**: Our `implicit_midpoint` function accepts as it's first input a function handle `f`. In Julia, functions are themselves objects and can be passed to other functions. For this problem, our function `f` will always be `pendulum_dynamics`, so you'd call your implicit midpoint method like: `implicit_midpoint(pendulum_dynamics, x1, x2, h)`

In [3]:
# TASK: Implement the `pendulum_dynamics` function
#  TIP: When allocating the xdot vector, be sure to use `zero(x)` to make it the same type as the input vector.
#       This makes it work with ForwardDiff, since `eltype(x)` will be a ForwardDiff.Dual type.
"""
    pendulum_dynamics(x; [m, l, g])

Returns the derivative of the state vector `x = [θ, ω]`, given mass `m`, length `l`, and gravity `g`.
"""
function pendulum_dynamics(x; m=1, l=1, g=9.81)
    # TODO: Implement this method
    xdot = zero(x)
    xdot[1] = x[2]
    xdot[2] = -(g/l)*sin(x[1])
    return xdot
end

# TASK: Implement the `implicit_midpoint_pendulum!` function
"""
    x2 = implicit_midpoint(f, x1, x2, h)

Evaluate the residual of the implicit midpoint integrator for the dynamics `f` with a time step of `h`.
The dynamics function `f` is expected to take a state vector as an argument. 
"""
function implicit_midpoint(f, x1, x2, h)
    # TODO: Implement this method
    out = zero(x2)
    out = x1 + h * f((x2+x1)/2)-x2
    return out
end

implicit_midpoint

In [4]:
@testset "1a" begin                                                            # POINTS = 3
    f = pendulum_dynamics
    x1 = [0.12, -1.5]
    x2 = [-0.5, 10.2]
    h = 0.1
    @testset "pendulum dynamics" begin                                         # POINTS = 1
        @test pendulum_dynamics([0,0.]) ≈ zeros(2)                             # POINTS = 0
        @test pendulum_dynamics(x1) ≈ load(resfile, "xdot")                    # POINTS = 1
    end
    @testset "implicit midpoint" begin                                         # POINTS = 2
        @test implicit_midpoint(f, zeros(2), zeros(2), h) ≈ zeros(2)           # POINTS = 0
        @test implicit_midpoint(f, x1, x1, 0) ≈ zeros(2)                       # POINTS = 1
        @test implicit_midpoint(f, x1, x2, h) ≈ load(resfile, "midpoint_res")  # POINTS = 1
    end
end;

LoadError: LoadError: UndefVarError: @testset not defined
in expression starting at In[4]:1

## Part (b): Solve using Newton's Method (10 pts)
Use Newton's method to solve for $x_{n+1}$ with initial guess $x_{n+1} = x_n$. Solve such that the 2-norm of the residual is less than 1e-6. 

Recall that Newton's method is an interative root-finding method. Each iteration looks like:

$$ x^{(i+1)} = x^{(i)} - \frac{\partial f}{\partial x}^{-1} f(x) $$

**NOTE**: It should take 5 iterations or less. If it's taking more iterations than that, you're doing something wrong.

In [4]:
# TASK: Complete the following function
using ForwardDiff
"""
    implicit_midpoint_solve!(f,x1,x2,h)

Find the `x2` that satisfies implicit midpoint integration for the pendulum dynamics. 

Should modify `x2` in-place and return a vector of 2-norm residuals. The input value of `x2`
should be used as the initial guess.
"""
function implicit_midpoint_solve!(f, x1, x2, h; ϵ=1e-6)::Vector{Float64}
    # TODO: Implement this function. Feel free to add optional input arguments as needed
    residuals = Float64[]
    x = [x2[1]
        x2[2]]
    diff = implicit_midpoint(f,x1,x,h)
    residuals = [norm(diff)]
    
    while norm(diff) > ϵ
        delt = implicit_midpoint(f,x1,x,h)
        J = ForwardDiff.jacobian(x->implicit_midpoint(f,x1,x,h),x)
        dx = -J \ delt
        x  = x + dx
        diff = implicit_midpoint(f,x1,x,h)
        residuals = [residuals norm(diff)]
    end
    x2[1] = x[1]
    x2[2] = x[2]
    
    return vec(residuals)
end

implicit_midpoint_solve!

In [5]:
@testset "1b" begin                                # POINTS = 10
    f = pendulum_dynamics
    x0 = [deg2rad(45), 0]
    h = 0.1
    @testset "Implicit Midpoint Solve" begin       # POINTS = 10
        x_next = copy(x0)
        residuals = implicit_midpoint_solve!(pendulum_dynamics, x0, x_next, h)
        @test residuals[end] < 1e-6                # POINTS = 4
        @test length(residuals) < 5                # POINTS = 1
        @test x_next ≈ load(resfile, "x_next")     # POINTS = 5
    end
end;

Test Summary: | Pass  Total
1b            |    3      3


## Part (c): Energy Analysis (2 pts)
Simulate with $h=0.1$ for 1 hour and plot the total energy of the system vs time:
$$ E = \frac{1}{2} m l^2 \omega^2 + m g l (1 - \cos \theta) $$

Start the simulation with an initial state of 45 degrees and zero velocity.

In [6]:
# TASK: Compute the energy of the system. Store the result in the vector `energy_implicit::Vector{Float64}`
using Plots


energy_implicit = Float64[]
x = [pi/4
    0]
f = pendulum_dynamics
t = Array(range(0,3600,step=0.1))
h = 0.1
g = 9.81
x_nxt = x

for i = 1:(length(t))
    residuals = implicit_midpoint_solve!(f,x,x_nxt,h)
    E = 0.5*1*x_nxt[2]*x_nxt[2]+g*1*(1-cos(x_nxt[1]))
    energy_implicit = [energy_implicit
                                    E]
    x = x_nxt
end

plot(t,energy_implicit)

LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to C:\Users\bdw19\.julia\compiled\v1.7\Plots\jl_D28F.tmp.

In [32]:
@testset "1c" begin                                                                  # POINTS = 2
    @test mean(energy_implicit) ≈ load(resfile, "implicit_energy_mean") atol = 1e-3  # POINTS = 1
    @test std(energy_implicit) < 0.05                                                # POINTS = 1
end;

1c: Error During Test at In[32]:2
  Test threw exception
  Expression: ≈(mean(energy_implicit), load(resfile, "implicit_energy_mean"), atol = 0.001)
  UndefVarError: energy_implicit not defined
  Stacktrace:
   [1] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:445 [inlined]
   [2] macro expansion
     @ In[32]:2 [inlined]
   [3] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:1283 [inlined]
   [4] top-level scope
     @ In[32]:2
1c: Error During Test at In[32]:3
  Test threw exception
  Expression: std(energy_implicit) < 0.05
  UndefVarError: energy_implicit not defined
  Stacktrace:
   [1] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:445 [inlined]
   [2] macro expansion
     @ In[32]:3 [inlined]
   [3] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 2 errored, 0 broken.[39m

## Part (d): RK4 Comparison (5 pts)
Compare the energy behavior of the implicit midpoint integrator with a 4th order Runge Kutta integrator.

In [7]:
# TASK: implement a 4th order Runge Kutta integrator for the pendulum (3 pts)
"""
    rk4(f, x, h)

Integrate the dynamics `f(x)` with a 4th Order Runge Kutta method at state vector `x` and time step `h`.
"""
function rk4(f, x, h)
    # TODO: implement
    xnext = zero(x)
    first = f(x)
    second = f(x+0.5*h*first)
    third = f(x+0.5*h*second)
    forth = f(x+h*third)
    xnext = x+(h/6)*(first+2*second+2*third+forth)
    
    return xnext
end

rk4

In [8]:
@testset "1d" begin                                # POINTS = 1
    @testset "rk4" begin                           # POINTS = 1
        f = pendulum_dynamics
        x = [-0.12, 1.37]
        h = 0.1
        @test rk4(f, x, h) ≈ load(resfile, "rk4")  # POINTS = 1
    end
end;

Test Summary: | Pass  Total
1d            |    1      1


In [9]:
# TASK: Compute the energy behavior of rk4. Store the result in `energy_rk4::Vector{Float64}` (1 pt)
# TASK: Generate a plot of energy vs time comparing implicit midpoint with rk4 (1 pt)

using Plots

energyrk4 = Float64[]
x = [pi/4
    0]
t = Array(range(0,3600,step=0.1))
h = 0.1
g = 9.81
x_nxt = x

for i = 1:length(t)
    x_nxt = rk4(pendulum_dynamics,x,h)
    E = 0.5*1*x_nxt[2]*x_nxt[2]+g*1*(1-cos(x_nxt[1]))
    energyrk4 = [energyrk4
                    E]
    x = x_nxt
end

LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to C:\Users\bdw19\.julia\compiled\v1.7\Plots\jl_5E07.tmp.

In [10]:
@testset "1d" begin                                                   # POINTS = 4
    @testset "energy comparison" begin                                # POINTS = 4
        @test length(energy_rk4) == length(energy_implicit)
        @test energy_rk4[end] < energy_implicit[end]                  # POINTS = 1
        @test mean(diff(energy_rk4)) ≈ -2.5e-5 atol=1e-5              # POINTS = 2
        @test std(diff(energy_rk4)) < 1e-3  # should be about linear  # POINTS = 1
    end
end;

energy comparison: Error During Test at In[10]:3
  Test threw exception
  Expression: length(energy_rk4) == length(energy_implicit)
  UndefVarError: energy_rk4 not defined
  Stacktrace:
   [1] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:445 [inlined]
   [2] macro expansion
     @ In[10]:3 [inlined]
   [3] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:1283 [inlined]
   [4] macro expansion
     @ In[10]:3 [inlined]
   [5] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:1283 [inlined]
   [6] top-level scope
     @ In[10]:2
energy comparison: Error During Test at In[10]:4
  Test threw exception
  Expression: energy_rk4[end] < energy_implicit[end]
  UndefVarError: energy_rk4 not defined
  Stacktrace:
   [1] macro expansion
     @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 4 errored, 0 broken.[39m

## Part (e): Evaluating the Jacobian (5 pts)
Computing the Jacobian for an explicit integrator like RK4 is straightforward, since it's just a basic application of the chain rule (you should do this for practice, and can check your result with ForwardDiff). 

Computing the Jacobian for an implicit integrator is not as trivial since we use Newton's method to compute the next step. However, we can use the [implicit function theorem](https://en.wikipedia.org/wiki/Implicit_function_theorem), which can be easily derived by taking a 1st order Taylor series expansion of $f(x,y) = 0$:
$$ f(x,y) = 0 \implies f(x + \Delta x, y + \Delta y) \approx f(x,y) + \frac{\partial f}{\partial x} \Delta x + \frac{\partial f}{\partial y} \Delta y = 0 $$
Dropping $f(x,y)$ (since it's zero) and solving for $\Delta x$ we obtain an expression for our Jacobian:
$$ \Delta x = -\frac{\partial f}{\partial x}^{-1} \frac{\partial f}{\partial y} \Delta y $$

Use this to compute the Jacobian of the implicit integrator, evaluated at $\theta_1 = \omega_1 = 0$ with $h = 0.1$. 

In [11]:
# TASK: Compute the Jacobian of the implicit midpoint integrator for h = 0.1. (3 pts)
#       Store the result in `Amid::Matrix{Float64}` of size (2,2)
Amid = zeros(2,2)
x = [0
    0]
h = 0.1
g = 9.81
dfdx = [1 -h/2
        g*h*0.5*cos(x[1]) 1]
dfdy = [-1 -h/2
        g*h*0.5*cos(x[1]) -1]
Amid = -dfdx\dfdy

2×2 Matrix{Float64}:
  0.952124  0.0976062
 -0.957517  0.952124

In [5]:
# TASK: Compute the magnitude of the eigenvalues of A for 0 ≤ h ≤ 1.  (1 pt)
#       Store the result in `eigs_implicit::Matrix{Float64}` of size (100,2)
eigs_implicit = zeros(100,2)
h = range(0,100,length=100)
for i = 1:1:100
    dfdx = [1 -h[i]/2
    g*h[i]*0.5*cos(x[1]) 1]
    dfdy = [-1 -h[i]/2
    g*h[i]*0.5*cos(x[1]) -1]
    Amid = -dfdx\dfdy
    eigs[i,:] = norm.(eigvals(Amid))
end
# TASK: Plot the eigenvalues vs time step (1 pt)
plot(h,eigs)

LoadError: UndefVarError: x not defined

Obviously something special is going on here. Implicit midpoint is a "symplectic" integrator. That means it conserves energy (up to numerical roundoff/truncation error). Implicit midpoint is also the simplest "collocation" method. More about those later...

In [49]:
@testset "1e" begin                               # POINTS = 5
    @test size(eigs_implicit) == (100,2)          # POINTS = 0
    @test mean(eigs_implicit) ≈ 1.0 atol=1e-6     # POINTS = 2
    @test std(eigs_implicit) ≈ 0 atol=1e-10       # POINTS = 2
    @test Amid ≈ load(resfile, "Amid") atol=1e-6  # POINTS = 1
end;


1e: Test Failed at In[49]:3
  Expression: ≈(mean(eigs_implicit), 1.0, atol = 1.0e-6)
   Evaluated: 0.0 ≈ 1.0 (atol=1.0e-6)
Stacktrace:
 [1] macro expansion
   @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:445 [inlined]
 [2] macro expansion
   @ In[49]:3 [inlined]
 [3] macro expansion
   @ C:\Users\bdw19\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\Test\src\Test.jl:1283 [inlined]
 [4] top-level scope
   @ In[49]:2
Test Summary: | Pass  Fail  Total
1e            |    3     1      4


LoadError: [91mSome tests did not pass: 3 passed, 1 failed, 0 errored, 0 broken.[39m